In [53]:
import os
import glob
import pydicom
import nibabel as nib
import pandas as pd
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../code/")

from utils.dcm_utils import *
from utils.nii_utils import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [59]:
DATA_DIR = "/Volumes/SSD970/"
TRAIN_SEG_DIR = os.path.join(DATA_DIR, "segmentations")
TRAIN_IMAGE_DIR = os.path.join(DATA_DIR, "train_images")

In [60]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, seg_dir, transform=None, target_transform=None):
        self.image_dir = image_dir
        self.seg_dir = seg_dir

        self.nii_files = glob.glob(seg_dir + "/*.nii")

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.nii_files)

    def __getitem__(self, idx):

        nii_path = self.nii_files[idx]
        UID = nii_path.split("/")[-1].replace(".nii", "")


        imgs, image_positions, image_orientations, pixel_spacings, slice_thicknesses, is_flip = read_patient_dcm(UID, self.image_dir)
        height = image_positions[0, 2] - image_positions[1, 2]
        pixel_spacing = pixel_spacings[0, 0]
        aspect = np.round(height / pixel_spacing)
        img3d = np.repeat(imgs, aspect, axis=0)

        # get segmentation
        seg = read_patient_nii_mask(UID, self.seg_dir)
        if is_flip is False:
            seg = np.flip(seg, axis=0)
        seg3d = np.repeat(seg, aspect, axis=0)

        if self.transform:
            img3d = self.transform(img3d)
        if self.target_transform:
            seg3d = self.target_transform(seg3d)
        return img3d, seg3d


dataset = SegmentationDataset(TRAIN_IMAGE_DIR, TRAIN_SEG_DIR)
print(len(dataset))
image, label = dataset[0]
print(image.shape)
print(label.shape)

87


TypeError: read_patient_dcm() takes 1 positional argument but 2 were given

In [38]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="resnet18",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights=None,     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

In [41]:
def convert_3d(module):

    new_module = module
    if isinstance(module, nn.Conv2d):
        new_module = nn.Conv3d(module.in_channels,
                               module.out_channels,
                               kernel_size=module.kernel_size[0],
                               stride=module.stride[0],
                               padding=module.padding[0],
                               bias=module.bias)
    elif isinstance(module, nn.BatchNorm2d):
        new_module = nn.BatchNorm3d(module.num_features,
                                    module.eps,
                                    module.affine,
                                    module.track_running_stats)
    elif isinstance(module, nn.MaxPool2d):
        new_module = nn.MaxPool3d(kernel_size=module.kernel_size,
                                  stride=module.stride,
                                  padding=module.padding,
                                  dilation=module.dilation,
                                  ceil_mode=module.ceil_mode)

    for name, child_module in new_module.named_children():
        setattr(new_module, name, convert_3d(child_module))

    return new_module

model_3d = convert_3d(model)
model_3d

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=True, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=True, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=True, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=

In [42]:
x = torch.randn(1, 1, 128, 128, 128)
out = model(x)
out.shape

torch.Size([1, 1, 128, 128, 128])

In [52]:
import numpy as np
from sklearn.model_selection import KFold


for group, (train_index, test_index) in enumerate(kf.split(X)):
    print("Group:", group, "TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


KFold(n_splits=5, random_state=None, shuffle=False)
Group: 0 TRAIN: [2 3 4 5 6 7 8 9] TEST: [0 1]
Group: 1 TRAIN: [0 1 4 5 6 7 8 9] TEST: [2 3]
Group: 2 TRAIN: [0 1 2 3 6 7 8 9] TEST: [4 5]
Group: 3 TRAIN: [0 1 2 3 4 5 8 9] TEST: [6 7]
Group: 4 TRAIN: [0 1 2 3 4 5 6 7] TEST: [8 9]


In [49]:
kf.split(X)

<generator object _BaseKFold.split at 0x7fdab5ce3c80>